# Generative Supervised Fine-tuning of GPT-2

Now that we have our GPT-2 model all trained up - we need a way we can get it to generate what we want.

In the following notebook, we're going to use an approach called "Supervised Fine-tuning" to achieve our goals today.

In essence, we're going to use each example as a self-contained unit (with potential for something called "packing") and this is going to allow us to build "labeled" data.

For this notebook, we're going to be flying quite high up in the levels of abstraction. Take extra care to look into the libraries we're using today!

Let's start by grabbing our dependencies, as always:



In [10]:
!pip install transformers accelerate datasets trl bitsandbytes -qU
#transformers library from hugging face
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dataset Curation

We're going to be fine-tuning our model on SQL generation today.

First thing we'll need is a dataset to train on!

We'll use [this](https://huggingface.co/datasets/b-mc2/sql-create-context) dataset today!

First up, let's load it and take a look at what we've got.

- [`load_dataset`](https://huggingface.co/docs/datasets/loading)

In [3]:
from datasets import load_dataset

sql_dataset = load_dataset(
   'b-mc2/sql-create-context' #Directly copied from https://huggingface.co/datasets/b-mc2/sql-create-context
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
sql_dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 78577
    })
})

In [16]:
### Display a Sample Row
index = np.random.randint(0,len(sql_dataset['train']))
print(index)
sql_dataset['train'][index]

49651


{'context': 'CREATE TABLE table_name_13 (byes INTEGER, losses VARCHAR, wins VARCHAR)',
 'answer': 'SELECT AVG(byes) FROM table_name_13 WHERE losses = 15 AND wins < 1',
 'question': 'What is the average number of Byes for the team that had 15 Losses, and less than 1 Win?'}

So, we've got ~78.5K rows of:

- question - a natural language query about
- context - the `CREATE TABLE` statement - which gives us important context about the table
- answer - a SQL query that is aligned with both the question and the context.

Let's split our data into `train`, `val`, and `test` datasets.

We can use our `train` and `val` sets to train and evaluate our model during training - and our `test` set to ultimately benchmark the generations of our model!

In [20]:
### YOUR CODE HERE (ensure your dataset looks similar the sample below)
test_size = 0.2
train_test = sql_dataset['train'].train_test_split(test_size=test_size)
train_test

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 62861
    })
    test: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 15716
    })
})

In [21]:
val_size = 0.5
test_val = train_test['test'].train_test_split(test_size=val_size)
test_val

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 7858
    })
})

In [25]:
from datasets import DatasetDict

split_sql_dataset = DatasetDict(
    {
    "train": train_test['train'],
    "val" : test_val['train'],
    "test" : test_val['test']
    }
)

split_sql_dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 62861
    })
    val: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 7858
    })
})

```
DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 62861
    })
    val: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 7858
    })
})
```

### Creating a "Prompt"

Now we need to create a prompt that's going to allow us to interact with our model when we desired the trained behaviour.

Think of this as a pattern that aligns the model with our desired outputs.

We need a single text prompt, as that is what the `SFTTrainer` we're going to use to fine-tune our model expects.

The basic idea is that we're going to merge the `question`, `context`, and `answer` into a single block of text that shows the model our desired outputs.

Let's look at what that block needs to look like:

```
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
{response}{eos_token}
```

Let's look at that from a completed prompt perspective to get a bit more information:

```
<|startoftext|>### Instruction:
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
You must output the SQL query that answers the question.

### Input:
How many locations did the team play at on week 7?

### Context:
CREATE TABLE table_24123547_2 (location VARCHAR, week VARCHAR)

### Response:\nSELECT COUNT(location) FROM table_24123547_2 WHERE week = 7<|endoftext|>
```

As you can see, our prompt contains completed examples of our task. We're going to show our model many of these examples over and over again to teach it to produce outputs that are aligned with our goals!

First step, let's create a template we can use to call `.format()` on while constructing our prompts.

In [28]:
### CREATE A FORMATTABLE PROMPT TEMPLATE
TEXT2SQL_TRAINING_PROMPT_TEMPLATE = """\
{bos_token}### Instruction:
{system_message}

### Input :
{input}

### Context :
{context}

### Response:
{response}{eos_token}"""

In [29]:
TEXT2SQL_INFERENCE_PROMPT_TEMPLATE = """\
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
"""

Now let's create a function we can map over our dataset to create the full prompt text block.

In [34]:
def create_sql_prompt(sample : dict):
  SYSTEM_MESSAGE = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
  You must output the SQL query that answers the question."""

  full_prompt = TEXT2SQL_TRAINING_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      eos_token = "<|endoftext|>",
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = sample["context"],
      response = sample["answer"],
  )

  return {"text" : full_prompt}

#### Helper Function Begin.

I've created this helper-function to be able to see how our model is doing visibly, rather than only through metrics.

In [35]:
def create_sql_prompt_and_response(sample):
  SYSTEM_MESSAGE = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
  You must output the SQL query that answers the question."""

  full_prompt = TEXT2SQL_INFERENCE_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = sample["context"]
  )

  ground_truth = sample["answer"]

  return {"full_prompt" : full_prompt, "ground_truth" : ground_truth}

#### Helper Function End.

Let's look at an example of a formatted prompt.

In [36]:
create_sql_prompt(split_sql_dataset["train"][0])

{'text': '<|startoftext|>### Instruction:\nYou are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.\n  You must output the SQL query that answers the question.\n\n### Input :\nFind the number of different states which banks are located at.\n\n### Context :\nCREATE TABLE bank (state VARCHAR)\n\n### Response:\nSELECT COUNT(DISTINCT state) FROM bank<|endoftext|>'}

Great!

Now we can map this over our dataset!

- [`DatasetDict.map()`](https://huggingface.co/docs/datasets/process#map)

In [37]:
split_sql_dataset = split_sql_dataset.map(create_sql_prompt)

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

In [38]:
split_sql_dataset['train'][0]

{'context': 'CREATE TABLE bank (state VARCHAR)',
 'answer': 'SELECT COUNT(DISTINCT state) FROM bank',
 'question': 'Find the number of different states which banks are located at.',
 'text': '<|startoftext|>### Instruction:\nYou are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.\n  You must output the SQL query that answers the question.\n\n### Input :\nFind the number of different states which banks are located at.\n\n### Context :\nCREATE TABLE bank (state VARCHAR)\n\n### Response:\nSELECT COUNT(DISTINCT state) FROM bank<|endoftext|>'}

## Load the Model And Preproccessing

Now for the moment we've all been waiting for...

Loading our model!

Let's use the `AutoModelForCausalLM` and `AutoTokenzier` classes from `transformers` to see just how easy this is.

- [`AutoModelForCausalLM`](https://huggingface.co/docs/transformers/v4.35.0/en/model_doc/auto#transformers.AutoModelForCausalLM)
- [`AutoTokenizer`](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoTokenizer)
- [GPT-2 Model Card](https://huggingface.co/gpt2)

In [40]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = 'gpt2'

gpt2_base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

gpt2_tokenizer = AutoTokenizer.from_pretrained(model_id)

We need to make sure our tokenizer has a `pad_token` in order to be able to pad sequences so they're all the same length.

We'll use a little trick here to set our padding token to our eos (end of sequence) token to make training go a little smoother.

In [41]:
# We have formatted our prompt according to this EOS token
# Was it done to be aligned to this tokenizer that we were about to use?
gpt2_tokenizer.eos_token

'<|endoftext|>'

In [42]:
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

In [44]:
gpt2_tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

We also need to make sure we resize our model to be aligned with the token embeddings. If we didn't do this - we'd face a shape error while training!

In [43]:
gpt2_base_model.resize_token_embeddings(len(gpt2_tokenizer))

Embedding(50257, 768)

Now let's use the Hugging Face `pipeline` to see what generation looks like for our untrained model.

In [45]:
from transformers import pipeline, set_seed, GenerationConfig
generator = pipeline('text-generation', model=gpt2_base_model, tokenizer=gpt2_tokenizer)
set_seed(42)

def generate_sample(sample):
  prompt_package = create_sql_prompt_and_response(sample)

  generation_config = GenerationConfig(
      max_new_tokens=50, #how many tokens we wish to generate
      do_sample=True,
      top_k=50,
      temperature=1e-4, #Temperature means creativity. Generally between 0 and 1
      eos_token_id=gpt2_base_model.config.eos_token_id,
  )

  generation = generator(prompt_package["full_prompt"], generation_config=generation_config)
  print("---------------")
  print("Model Response:")
  print(generation[0]["generated_text"].replace(prompt_package["full_prompt"], ""))
  print("+++++++++++++++")
  print("Ground Truth")
  print(prompt_package["ground_truth"])

In [46]:
generate_sample(split_sql_dataset["test"][0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Model Response:

You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

### Input:

What is the pole position of paul
+++++++++++++++
Ground Truth
SELECT pole_position FROM table_1140078_2 WHERE location = "Paul Ricard"


## Training the Model

Now that we have our model set up, our tokenizer set up, we can finally begin training!

Let's look at our Trainer, and set some hyper-parameters:

- `per_device_train_batch_size` - this is a batch size that accomodates distributed training - a default we could use is `4`
- `gradient_accumulation_steps` - this is exactly the same as the previous notebook, it's a way to "simulate" a large batch size by collecting losses over multiple iterations - scaling them - and then combining them together. - a default we could use is `4`
- `gradient_checkpointing` - I'll let the authors speak for themselves [here](https://github.com/cybertronai/gradient-checkpointing). In essence: This saves memory at the cost of computational time. - let's set this to `True`
- `max_grad_norm` - this is the value used for gradient clipping, which is a method of reducing vanishing gradient potential - let's use `0.3`
- `max_steps` - how many steps will we train for? - this is up to you
- `learning_rate` - how fast should we learn? - lets use `2e-4`
- `save_total_limit` - how many versions of the model will we save? - the default of `3` should work well
- `logging_steps` - how often we should log - up to you
- `output_dir` - where to save our checkpoints - up to you
- `optim` - which optimizer to use, you'll notice we're using a full precision paged optimizer - this is a performative and stable optimizer - but it uses extra memory - we should use `paged_adamw_32bit`
- `lr_scheduler_type` - we are once again using a cosine scheduler! - we should use `cosine`
- `evaluation_strategy` - we have an evaluation dataset, this defines when we should leverage it during training - we should use `steps`
- `eval_steps` - how many steps we should evaluate for - up to you
- `warmup_ration` - how many "warmup" steps we take to reach our full learning rate before we start decaying. This is a ration of our max_steps - the default value of `0.3` should work!

- [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.15.0/main_classes/trainer#transformers.TrainingArguments)

In [48]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    max_steps= 500,
    learning_rate = 2e-4,
    save_total_limit= 3,
    logging_steps= 10,
    output_dir="sql_gpt2",
    optim="paged_adamw_32bit",
    lr_scheduler_type= "cosine",
    evaluation_strategy  = "steps",
    eval_steps = 50,
    warmup_ratio = 0.05
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Now, for our `SFTTrainer` AKA "Where the magic happens".

This `SFTTrainer` is going to take our above training arguments, our data, our model and our tokenizer, and train it all for us!

Notice that we're setting `max_seq_length` to the maximum context window of our model - this ensures we do not exceed our maximum context window, and will pad our examples up to the maximum context window!

#### ❓QUESTION❓

What is the maximum input sequence length for GPT-2?

In [50]:
gpt2_tokenizer.max_model_input_sizes

{'gpt2': 1024,
 'gpt2-medium': 1024,
 'gpt2-large': 1024,
 'gpt2-xl': 1024,
 'distilgpt2': 1024}

In [51]:
trainer = SFTTrainer(
 gpt2_base_model,
 dataset_text_field="text",
 train_dataset=split_sql_dataset["train"],
 eval_dataset=split_sql_dataset["val"],
 tokenizer=gpt2_tokenizer,
 max_seq_length=1024,
 args=training_args
)

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

Finally, we can call our `.train()` method and watch it go!

In [52]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
50,0.772900,0.674925
100,0.660500,0.627899
150,0.641000,0.598317
200,0.594300,0.575556
250,0.575900,0.562851
300,0.555700,0.551588
350,0.551800,0.544605
400,0.536900,0.539101
450,0.564100,0.536022
500,0.566400,0.535654


TrainOutput(global_step=500, training_loss=0.6776670436859131, metrics={'train_runtime': 1678.2074, 'train_samples_per_second': 4.767, 'train_steps_per_second': 0.298, 'total_flos': 656681992704000.0, 'train_loss': 0.6776670436859131, 'epoch': 0.13})

Let's save our fine-tuned model!

In [53]:
trainer.save_model()

## Testing our Model

Now that we have a fine-tuned model, let's see how it did

In [54]:
ft_gpt2_model = AutoModelForCausalLM.from_pretrained("sql_gpt2")

In [55]:
generator = pipeline('text-generation', model=ft_gpt2_model, tokenizer=gpt2_tokenizer, )

In [56]:
generate_sample(split_sql_dataset["test"][0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Model Response:
SELECT pole_position FROM table_1140078_2 WHERE location = "Paul Ricard" AND location = "Paul Ricard" AND location = "Paul Ricard" AND location = "Paul Ricard" AND location
+++++++++++++++
Ground Truth
SELECT pole_position FROM table_1140078_2 WHERE location = "Paul Ricard"


In [60]:
random_index = np.random.randint(len(split_sql_dataset["test"]))
print(random_index)
generate_sample(split_sql_dataset["test"][random_index])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


5390
---------------
Model Response:
SELECT SUM(top_division_titles) FROM table_name_90 WHERE founded < 1975 AND location = "chaguaramas" AND year = "prize" AND year = "prize" AND year = "prize"
+++++++++++++++
Ground Truth
SELECT SUM(top_division_titles) FROM table_name_90 WHERE founded < 1975 AND location = "chaguaramas"


That is *significantly* better.

#### ❓QUESTION❓

What methods could we use to validate our SQL outputs?

1. One way to 'validate' our SQL outputs would be to simply query a database using the generated SQL and putting a check if the output is valid or not. Specially if an exception is thrown, that definitely means SQL is not vaid. It would be sequential, 1 by 1. Final evaluation could be the number of correct hits

2. Maybe some kind of SQL compiler?!

3. Using OpenAI's api and prompting 'IS this SQL valid' for each generated Sequence, lol!


#### ❓QUESTION❓

How would you extend this notebook to another use-case?

I would say this could NB can be extended any Supervised Fine tuning Generative task, where we want to teach GPT the prediction of a (post)sequence, given a (pre)sequence.
Github/MSFT's copilot definitely is one such 'coding' specific use case, where if we could train the model on million of lines of code, that developers generally write AND we can predict the next sequence of code, given an acceptable lenght of sequence of code.